In [58]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel, BertTokenizer, AutoModelForSequenceClassification
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import os
# from langchain.llms import OpenAI
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
import pickle
from transformers import pipeline
from sklearn.model_selection import train_test_split
from datasets import Dataset









In [20]:
file_path = 'data.csv'
df = pd.read_csv(file_path)
print("Initial Data:")
display(df.head())

Initial Data:


,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,"[-0.01590039,-0.034130897,0.005732614,-0.01853...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,"[-0.017608976,-0.021786924,0.01547983,-0.00932...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,"[0.00841488,-0.023665192,0.006762431,-0.013723...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,"[-0.012671886,-0.0039057182,0.019575326,-0.016...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,"[-0.015486176,-0.0125719,-0.0122843925,-0.0343...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [21]:
file_path = 'Data_latih.csv'
df2 = pd.read_csv(file_path)
print("Initial Data:")
display(df2.head())

Initial Data:


,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [22]:
df2 = df2.rename(columns={
    'ID': 'record_id',
    'judul': 'title',
    'narasi': 'content',
    'tanggal': 'publication_date'
})

display(df2.head())

,record_id,label,publication_date,title,content,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [23]:
df2 = df2.drop(columns=['nama file gambar', 'label'])
display(df2.head())

,record_id,publication_date,title,content
0,71,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...
1,461,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
2,495,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
3,550,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
4,681,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .


In [24]:
df2 = df2.iloc[1:].reset_index(drop=True)
display(df2.head())

,record_id,publication_date,title,content
0,461,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
1,495,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...
2,550,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
3,681,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .
4,736,17-Jun-20,event promo smartphone JNE 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma..."


In [26]:
columns_to_drop = ['image', 'url', 'embedding', 'created_at', 'updated_at','summary','source']
df.drop(columns=columns_to_drop, inplace=True)
print("\nAfter Dropping Unnecessary Columns:")
display(df.head())

KeyError: "['image', 'url', 'embedding', 'created_at', 'updated_at', 'summary', 'source'] not found in axis"

In [33]:
columns_rename_map = {
    'id': 'record_id',
    'source': 'news_source',
    'date': 'publication_date'
}
df.rename(columns=columns_rename_map, inplace=True)
print("\nAfter Renaming Columns:")
display(df.head())



After Renaming Columns:


,record_id,title,content,publication_date,status
0,83,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,1
1,84,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,1
2,85,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,1
3,86,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,1
4,87,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,1


In [ ]:
df.head()

,record_id,title,content,publication_date
0,83,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...","TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00
1,84,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00
2,85,HNW Mendukung Jamaah Umroh First Travel Dapatk...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00
3,86,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00
4,87,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00


In [27]:
def roman_to_int(roman):
    roman_map = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 
        'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 
        'IX': 9, 'X': 10
    }

    return roman_map.get(roman, roman)


def convert_roman_to_int(text):
    def replace_roman(match):
        roman_numeral = match.group(0)
        return str(roman_to_int(roman_numeral))  # Convert Roman numeral to integer
    

    return re.sub(r'\b(I|II|III|IV|V|VI|VII|VIII|IX|X)\b', replace_roman, text)


for col in ['title', 'content']:
    df[col] = df[col].apply(lambda x: convert_roman_to_int(x) if isinstance(x, str) else x)
    df[col] = df[col].str.lower()  # Convert to lowercase


display(df.head())


,id,title,content,date
0,83,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00
1,84,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00
2,85,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00
3,86,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00
4,87,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00


In [29]:
def roman_to_int(roman):
    roman_map = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 
        'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 
        'IX': 9, 'X': 10
    }

    return roman_map.get(roman, roman)


def convert_roman_to_int(text):
    def replace_roman(match):
        roman_numeral = match.group(0)
        return str(roman_to_int(roman_numeral))  # Convert Roman numeral to integer
    

    return re.sub(r'\b(I|II|III|IV|V|VI|VII|VIII|IX|X)\b', replace_roman, text)


for col in ['title', 'content']:
    df2[col] = df2[col].apply(lambda x: convert_roman_to_int(x) if isinstance(x, str) else x)
    df2[col] = df2[col].str.lower()  # Convert to lowercase


display(df2.tail())


,record_id,publication_date,title,content
4225,999287,17-Jan-16,: kpk dilarang membawa brimob bersenjata masuk...,kpk dilarang membawa brimob bersenjata masuk g...
4226,999472,20-Aug-20,foto pejabat keuangan dibawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...
4227,999501,17-Aug-20,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny
4228,999848,21-Jul-20,kaesang: bapak saya dengan kesederhaan bisa ni...,bapak saya dengan kesederhaan bisa nipu rakyat...
4229,999860,20-Jul-20,laser dari termometer gun akan merusak struktu...,"in: saya nolak. kalau anda mau periksa, bukan ..."


In [30]:
df['status'] = 1
df2['status'] = 0
print(df)
print(df2)

          id                                              title  \
0         83  depo plumpang terbakar, anggota dpr minta pert...   
1         84  jokowi perintahkan wapres ma'ruf amin tinjau l...   
2         85  hnw mendukung jamaah umroh first travel dapatk...   
3         86  tim dokkes polri telah terima 14 kantong jenaz...   
4         87  bamsoet ajak komunitas otomotif kembangkan per...   
...      ...                                                ...   
32730  63563  5 hewan aneh yang hidup di palung mariana, sal...   
32731  63564  biadab! guru sd di banyuwangi cabuli siswinya,...   
32732  63565  aji santoso: jakmania perlakukan persebaya den...   
32733  63566  sopir audi cium kaki ibu & istri hamil usai si...   
32734  63567          ini besaran biaya haji 2023 per embarkasi   

                                                 content  \
0      tempo.co, jakarta - anggota komisi 7 dpr ri ro...   
1      tempo.co, jakarta - presiden joko widodo atau ...   
2      info nas

In [34]:
df_merge = pd.merge(df, df2, on=['record_id', 'content','status','title','publication_date'], how='outer')
df_merge.tail()


,record_id,title,content,publication_date,status
36960,999287,: kpk dilarang membawa brimob bersenjata masuk...,kpk dilarang membawa brimob bersenjata masuk g...,17-Jan-16,0
36961,999472,foto pejabat keuangan dibawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...,20-Aug-20,0
36962,999501,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny,17-Aug-20,0
36963,999848,kaesang: bapak saya dengan kesederhaan bisa ni...,bapak saya dengan kesederhaan bisa nipu rakyat...,21-Jul-20,0
36964,999860,laser dari termometer gun akan merusak struktu...,"in: saya nolak. kalau anda mau periksa, bukan ...",20-Jul-20,0


In [35]:
df_merge = df_merge.dropna()
print(df_merge.isnull().sum().sum())


0


In [36]:
cleaned_file_path = 'cleaned_merge_data.csv'
df_merge.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned data saved to {cleaned_file_path}")
df_merge.tail()


Cleaned data saved to cleaned_merge_data.csv


,record_id,title,content,publication_date,status
36960,999287,: kpk dilarang membawa brimob bersenjata masuk...,kpk dilarang membawa brimob bersenjata masuk g...,17-Jan-16,0
36961,999472,foto pejabat keuangan dibawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...,20-Aug-20,0
36962,999501,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny,17-Aug-20,0
36963,999848,kaesang: bapak saya dengan kesederhaan bisa ni...,bapak saya dengan kesederhaan bisa nipu rakyat...,21-Jul-20,0
36964,999860,laser dari termometer gun akan merusak struktu...,"in: saya nolak. kalau anda mau periksa, bukan ...",20-Jul-20,0


In [37]:
print(df_merge.isnull().sum().sum())


0


In [38]:

cleaned_file_path = 'cleaned_merge_data.csv'
df = pd.read_csv(cleaned_file_path)

columns_to_tokenize = ['title', 'content']

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

for column in columns_to_tokenize:
    df[f"{column}_tokens"] = df[column].apply(
        lambda x: tokenizer.tokenize(x) if pd.notnull(x) else []
    )
    df[f"{column}_token_ids"] = df[column].apply(
        lambda x: tokenizer.encode(x, truncation=True, max_length=512) if pd.notnull(x) else []
    )

tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)
print(f"Tokenized data saved to {tokenized_file_path}")


KeyboardInterrupt: 

In [41]:
cleaned_file_path = 'cleaned_merge_data.csv'
df = pd.read_csv(cleaned_file_path)

# Columns to tokenize
columns_to_tokenize = ['title', 'content']

# Load IndoBERT tokenizer (IndoBERT is based on BERT, but fine-tuned for Indonesian)
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

# Tokenize and encode the columns
for column in columns_to_tokenize:
    # Tokenize the text (splitting into subword tokens)
    df[f"{column}_tokens"] = df[column].apply(
        lambda x: tokenizer.tokenize(x) if pd.notnull(x) else []
    )
    
    # Encode the text into token IDs (including truncation and padding)
    df[f"{column}_token_ids"] = df[column].apply(
        lambda x: tokenizer.encode(x, truncation=True, max_length=512) if pd.notnull(x) else []
    )

# Save the tokenized data to a new CSV file
tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)

# Output message
print(f"Tokenized data saved to {tokenized_file_path}")


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Raja\.cache\huggingface\hub\models--indobenchmark--indobert-base-p2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Tokenized data saved to tokenized_data.csv


In [65]:
cleaned_file_path = 'cleaned_merge_data.csv'
df = pd.read_csv(cleaned_file_path)

columns_to_tokenize = ['title', 'content']

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")

for column in columns_to_tokenize:
    df[f"{column}_tokens"] = df[column].apply(
        lambda x: tokenizer.tokenize(x) if pd.notnull(x) else []
    )
    
    df[f"{column}_token_ids"] = df[column].apply(
        lambda x: tokenizer.encode(x, truncation=True, max_length=512) if pd.notnull(x) else []
    )

tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors


In [66]:
tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)
df.head()

,record_id,title,content,publication_date,status,title_tokens,title_token_ids,content_tokens,content_token_ids
0,83,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,1,"[▁de, po, ▁plu, mpang, ▁terbakar, ,, ▁anggota,...","[0, 8, 771, 12266, 58098, 215404, 4, 26881, 10...","[▁tempo, ., co, ,, ▁ja, karta, ▁-, ▁anggota, ▁...","[0, 2167, 5, 587, 4, 79, 59505, 20, 26881, 602..."
1,84,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,1,"[▁jo, kowi, ▁perintah, kan, ▁wa, pres, ▁ma, ',...","[0, 741, 62248, 95447, 331, 259, 9518, 291, 25...","[▁tempo, ., co, ,, ▁ja, karta, ▁-, ▁presiden, ...","[0, 2167, 5, 587, 4, 79, 59505, 20, 79390, 625..."
2,85,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,1,"[▁h, n, w, ▁mendukung, ▁ja, ma, ah, ▁um, roh, ...","[0, 1096, 19, 434, 80279, 79, 192, 1366, 286, ...","[▁info, ▁nasional, ▁-, ▁wakil, ▁ketua, ▁mp, r,...","[0, 3004, 47951, 20, 81724, 90558, 9607, 42, 1..."
3,86,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,1,"[▁tim, ▁dok, kes, ▁pol, ri, ▁telah, ▁terima, ▁...","[0, 5527, 11081, 9248, 3722, 416, 1825, 38119,...","[▁tempo, ., co, ,, ▁ja, karta, ▁-, ▁tim, ▁ke, ...","[0, 2167, 5, 587, 4, 79, 59505, 20, 5527, 311,..."
4,87,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,1,"[▁bam, so, et, ▁a, jak, ▁komunitas, ▁oto, moti...","[0, 81020, 991, 126, 10, 9860, 152718, 21668, ...","[▁info, ▁nasional, ▁-, ▁ketua, ▁mp, r, ▁ri, ▁s...","[0, 3004, 47951, 20, 90558, 9607, 42, 1427, 70..."


In [67]:
def reconstruction_accuracy(df, columns_to_tokenize, tokenizer):
    matches = 0
    total = 0

    for column in columns_to_tokenize:
        for text in df[column]:
            tokens = tokenizer.tokenize(text)
            
            reconstructed = tokenizer.convert_tokens_to_string(tokens)
            
            if text == reconstructed:
                matches += 1
            
            total += 1

    return matches / total if total > 0 else 0.0

columns_to_tokenize = ['title', 'content']
accuracy = reconstruction_accuracy(df, columns_to_tokenize, tokenizer)
print(f"Total reconstruction accuracy: {accuracy:.2%}")


Total reconstruction accuracy: 81.19%


In [ ]:
for column in columns_to_tokenize:
    for text in df[column].head():  
        tokens = tokenizer.tokenize(text)
        reconstructed = tokenizer.convert_tokens_to_string(tokens)
        print(f"Original: {text}")
        print(f"Tokens: {tokens}")
        print(f"Reconstructed: {reconstructed}")


Original: depo plumpang terbakar, anggota dpr minta pertamina pastikan pasokan bbm tak terganggu
Tokens: ['▁de', 'po', '▁plu', 'mpang', '▁terbakar', ',', '▁anggota', '▁d', 'pr', '▁minta', '▁per', 'tamina', '▁pastikan', '▁paso', 'kan', '▁', 'bb', 'm', '▁tak', '▁ter', 'ganggu']
Reconstructed: depo plumpang terbakar, anggota dpr minta pertamina pastikan pasokan bbm tak terganggu
Original: jokowi perintahkan wapres ma'ruf amin tinjau lokasi kebakaran depo plumpang
Tokens: ['▁jo', 'kowi', '▁perintah', 'kan', '▁wa', 'pres', '▁ma', "'", 'ruf', '▁amin', '▁tin', 'jau', '▁lokasi', '▁kebakaran', '▁de', 'po', '▁plu', 'mpang']
Reconstructed: jokowi perintahkan wapres ma'ruf amin tinjau lokasi kebakaran depo plumpang
Original: hnw mendukung jamaah umroh first travel dapatkan haknya
Tokens: ['▁h', 'n', 'w', '▁mendukung', '▁ja', 'ma', 'ah', '▁um', 'roh', '▁first', '▁travel', '▁dapatkan', '▁hak', 'nya']
Reconstructed: hnw mendukung jamaah umroh first travel dapatkan haknya
Original: tim dokkes polri te

In [71]:
import torch
import pandas as pd
from tqdm import tqdm  # Progress bar
from transformers import AutoTokenizer, AutoModel

# Load tokenized data
tokenized_file_path = 'tokenized_data.csv'
df = pd.read_csv(tokenized_file_path)

# Columns to process
columns_to_tokenize = ['title_token_ids', 'content_token_ids']

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")
model.eval()  # Set model to evaluation mode

# Embedding extraction function with progress bar
def get_embeddings_from_token_ids(token_ids_column):
    embeddings = []
    
    # Wrap the loop with tqdm to display a progress bar
    for token_ids in tqdm(token_ids_column, desc="Generating embeddings"):
        if pd.notnull(token_ids) and len(token_ids) > 0: 
            token_ids = eval(token_ids)  # Convert stringified list back to list of integers
            
            inputs = torch.tensor([token_ids])
            
            with torch.no_grad():
                outputs = model(inputs)
                last_hidden_state = outputs.last_hidden_state

                # Compute the mean of hidden states across the sequence
                embedding = last_hidden_state.mean(dim=1).squeeze().numpy()
                embeddings.append(embedding)
        else:
            embeddings.append([])  # Append an empty list if token IDs are missing
    
    return embeddings

# Process columns with progress bars
for column in columns_to_tokenize:
    print(f"Processing column: {column}")
    df[f"{column}_embeddings"] = get_embeddings_from_token_ids(df[column])

# Save the updated DataFrame
tokenized_with_embeddings_file_path = 'tokenized_with_embeddings.csv'
df.to_csv(tokenized_with_embeddings_file_path, index=False)

print("Embeddings have been successfully added to the DataFrame.")


Processing column: title_token_ids


Generating embeddings: 100%|██████████| 36524/36524 [1:03:52<00:00,  9.53it/s] 


Processing column: content_token_ids


Generating embeddings: 100%|██████████| 36524/36524 [6:48:54<00:00,  1.49it/s]  


Embeddings have been successfully added to the DataFrame.


In [ ]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

df_merge['combined_text'] = df_merge['title'] + " " + df_merge['content'] + " " + df_merge['summary']
df_merge['combined_text'] = df_merge['combined_text'].apply(lambda x: str(x) if pd.notna(x) else "")


text_embeddings = model.encode(df_merge['combined_text'].tolist(), batch_size=32, show_progress_bar=True)

np.save('embeddings.npy', text_embeddings)

print(text_embeddings.shape)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\adiat\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adiat\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1156 [00:00<?, ?it/s]

(36964, 384)


In [15]:
os.environ["OPENAI_API_KEY"] = "Kalo mo pake masukin API key kalian bang"


In [16]:
# df_merge['input'] = df_merge.apply(
#     lambda x: f"title: {' '.join(x['title_tokens'])} content: {' '.join(x['content_tokens'])} summary: {' '.join(x['summary_tokens'])}", axis=1
# )

# llm = OpenAI(model_name="gpt-4", temperature=0)

# prompt = PromptTemplate(
#     input_variables=["input"],
#     template="Analyze the following news and determine its authenticity: {input}"
# )

# llm_chain = LLMChain(llm=llm, prompt=prompt)

# def analyze_row(row):
#     try:
#         return llm_chain.run({"input": row['input']})
#     except Exception as e:
#         print(f"Error processing row: {e}")
#         return "Error"

# batch_size = 200

# for start in range(0, len(df_merge), batch_size):
#     batch = df_merge.iloc[start:start+batch_size]
#     batch['predicted_label'] = batch.apply(analyze_row, axis=1)
    
#     batch.to_csv(f"llm_predictions_batch_{start}.csv", index=False)
#     print(f"Processed batch {start}-{start + batch_size - 1}")

# print("Batch processing completed and saved.")

In [17]:
embeddings = np.load("embeddings.npy").astype('float32')

index = faiss.IndexFlatL2(embeddings.shape[1])  

index.add(embeddings)

print(f"Index contains {index.ntotal} documents")

Index contains 36964 documents


In [ ]:
embeddings = np.load("embeddings.npy").astype('float32')
documents = df_merge['content'].tolist()  # Document content
metadata = [{"title": row['title'], "summary": row['summary']} for _, row in df_merge.iterrows()]  # Metadata for documents

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

docstore_data = {str(i): {"page_content": documents[i], "metadata": metadata[i]} for i in range(len(documents))}
docstore = InMemoryDocstore(docstore_data)

index_to_docstore_id = {i: str(i) for i in range(len(documents))}

faiss.write_index(index, "faiss_index.index.faiss")

with open("docstore.pkl", "wb") as f:
    pickle.dump(docstore, f)
with open("index_to_docstore_id.pkl", "wb") as f:
    pickle.dump(index_to_docstore_id, f)

print("FAISS index, docstore, and mappings saved.")





FAISS index, docstore, and mappings saved.


In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

documents = []
with open("documents.txt", "r") as f:
    documents = f.readlines()

embeddings = np.load("document_embeddings.npy")
index = faiss.read_index("document_index.index")

def retrieve_similar_documents(query, top_k=3):
    query_embedding = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        query_output = model(**query_embedding)
    query_embedding = query_output.last_hidden_state.mean(dim=1).numpy()

    D, I = index.search(query_embedding, top_k)  
    similar_documents = [documents[idx] for idx in I[0]]
    return similar_documents

query = "AI technology is advancing rapidly, how is it affecting jobs?"
similar_docs = retrieve_similar_documents(query)

print("Top 3 similar documents:")
for doc in similar_docs:
    print(doc)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\adiat\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adiat\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Top 3 similar documents:
Another document about machine learning.
This is a document about AI.

Another document about machine learning.


In [20]:

fake_news_classifier = pipeline("text-classification", model="indobenchmark/indobert-base-p1")

def integrate_for_fake_news_detection(query, similar_documents):
    context = " ".join(similar_documents)  
    combined_input = f"Query: {query} Context: {context}"

    result = fake_news_classifier(combined_input)
    
    return result

result = integrate_for_fake_news_detection(query, similar_docs)
print("Fake News Detection Result:", result)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

c:\Users\adiat\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adiat\.cache\huggingface\hub\models--indobenchmark--indobert-base-p1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Fake News Detection Result: [{'label': 'LABEL_0', 'score': 0.34845447540283203}]


In [10]:
import pandas as pd
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [13]:

df = pd.read_csv('cleaned_data.csv')

# Replace NaN values in 'content' column with empty strings
df['content'] = df['content'].fillna('')

# Initialize the IndoBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')


def preprocess_data(text):
   
    if not isinstance(text, str):
        text = str(text)  
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

# Apply the preprocessing to the 'content' or 'title' column
df['inputs'] = df['content'].apply(preprocess_data)

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['inputs'].tolist(), df['Status'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# import pandas as pd
# import torch
# from sklearn.model_selection import train_test_split

# max_length = 128

# # Load the datasets
# df_hoax = pd.read_csv('tokenized_data_hoax.csv')
# df = pd.read_csv('tokenized_data.csv')

# # Ensure the 'Status' column is in an appropriate format (e.g., 0 and 1 for binary classification)
# df_hoax['Status'] = df_hoax['Status'].apply(lambda x: 1 if x == 'fake' else 0)  # Assuming 'fake' is the hoax label
# df['Status'] = df['Status'].apply(lambda x: 1 if x == 'fake' else 0)  # Adjust if needed

# # Split into train and test sets (using df_hoax as an example)
# train_hoax, test_hoax = train_test_split(df_hoax, test_size=0.2, random_state=42)

# # Use the tokenized columns (title_token_ids and content_token_ids) directly
# train_title_tokens = train_hoax['title_token_ids'].apply(eval).tolist()  
# train_content_tokens = train_hoax['content_token_ids'].apply(eval).tolist()

# test_title_tokens = test_hoax['title_token_ids'].apply(eval).tolist()
# test_content_tokens = test_hoax['content_token_ids'].apply(eval).tolist()

# # Convert token lists into PyTorch tensors
# train_title_tokens_tensor = torch.tensor(train_title_tokens)
# train_content_tokens_tensor = torch.tensor(train_content_tokens)

# test_title_tokens_tensor = torch.tensor(test_title_tokens)
# test_content_tokens_tensor = torch.tensor(test_content_tokens)

# # Labels as tensors
# train_labels = torch.tensor(train_hoax['Status'].values)
# test_labels = torch.tensor(test_hoax['Status'].values)

# train_inputs = torch.cat((train_title_tokens_tensor, train_content_tokens_tensor), dim=1)
# test_inputs = torch.cat((test_title_tokens_tensor, test_content_tokens_tensor), dim=1)

ValueError: expected sequence of length 18 at dim 1 (got 20)

In [19]:
# Load the datasets
df_hoax = pd.read_csv('tokenized_data_hoax.csv')
df = pd.read_csv('tokenized_data.csv')

# Ensure the 'Status' column is in an appropriate format (e.g., 0 and 1 for binary classification)
df_hoax['Status'] = df_hoax['Status'].apply(lambda x: 1 if x == 'fake' else 0)  # Assuming 'fake' is the hoax label
df['Status'] = df['Status'].apply(lambda x: 1 if x == 'fake' else 0)  # Adjust if needed

# Split into train and test sets (using df_hoax as an example)
train_hoax, test_hoax = train_test_split(df_hoax, test_size=0.2, random_state=42)

# Use the tokenized columns (title_token_ids and content_token_ids) directly
train_title_tokens = train_hoax['title_token_ids'].apply(eval).tolist()  
train_content_tokens = train_hoax['content_token_ids'].apply(eval).tolist()

test_title_tokens = test_hoax['title_token_ids'].apply(eval).tolist()
test_content_tokens = test_hoax['content_token_ids'].apply(eval).tolist()

# Define maximum lengths for padding
max_title_length = 50  # Adjust based on your data
max_content_length = 200  # Adjust based on your data

# Function to pad or truncate sequences
def pad_or_truncate(tokens, max_length):
    if len(tokens) > max_length:
        return tokens[:max_length]
    else:
        return tokens + [0] * (max_length - len(tokens))

# Apply padding/truncation
train_title_tokens_padded = [pad_or_truncate(tokens, max_title_length) for tokens in train_title_tokens]
train_content_tokens_padded = [pad_or_truncate(tokens, max_content_length) for tokens in train_content_tokens]

test_title_tokens_padded = [pad_or_truncate(tokens, max_title_length) for tokens in test_title_tokens]
test_content_tokens_padded = [pad_or_truncate(tokens, max_content_length) for tokens in test_content_tokens]

# Convert token lists into PyTorch tensors
train_title_tokens_tensor = torch.tensor(train_title_tokens_padded)
train_content_tokens_tensor = torch.tensor(train_content_tokens_padded)

test_title_tokens_tensor = torch.tensor(test_title_tokens_padded)
test_content_tokens_tensor = torch.tensor(test_content_tokens_padded)

# Labels as tensors
train_labels = torch.tensor(train_hoax['Status'].values)
test_labels = torch.tensor(test_hoax['Status'].values)

# Concatenate title and content tensors along the sequence dimension
train_inputs = torch.cat((train_title_tokens_tensor, train_content_tokens_tensor), dim=1)
test_inputs = torch.cat((test_title_tokens_tensor, test_content_tokens_tensor), dim=1)


In [ ]:
# Load the pre-trained IndoBERT model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",  
    save_strategy="epoch",        
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True   
)

# Trainer definition
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings,
    tokenizer=tokenizer,
)

# Start training
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [ ]:
# Save the trained model (kalau udh bagus)
# model.save_pretrained('./fake_news_model')
# tokenizer.save_pretrained('./fake_news_model')


In [ ]:
# Evaluate the model on test data
predictions = trainer.predict(test_encodings)
pred_labels = torch.argmax(predictions.predictions, axis=-1).item()

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Compute the evaluation metrics
accuracy = accuracy_score(test_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, pred_labels, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
# # Load the fine-tuned model
# model = BertForSequenceClassification.from_pretrained('./fake_news_model')
# tokenizer = BertTokenizer.from_pretrained('./fake_news_model')

# # Sample query
# query = "Apakah berita ini benar atau hoax?"
# inputs = tokenizer(query, return_tensors="pt")

# # Make prediction
# outputs = model(**inputs)
# logits = outputs.logits
# predicted_label = torch.argmax(logits, dim=-1).item()

# print(f"Predicted label: {predicted_label}")
# Untuk kalau udh dapat model yang bagus 
# can use it to make predictions on new data inserted